In [1]:
import os
import time
import random
import threading
import tempfile
from collections import defaultdict


class DiskIOBenchmark:
    def __init__(self):
        self.size = 1
        self.benchmarks = defaultdict(list)
        self.fp = os.open("./test.txt", os.O_CREAT|os.O_RDWR|os.O_SYNC | os.O_TRUNC, 0o777)
        self.sz = os.write(self.fp, bytearray(os.urandom(1 << 30)))
        self._setup()

    def _setup(self):
        self.is_open = True
        self.io = 0
        self.duration = 10

    def compute_throughput(self):
        for _ in range(self.duration):
            time.sleep(1)
            self.benchmarks[self.size].append(self.io)
            self.io = 0
        self.is_open = False

    def random_writes(self):
        k = bytearray(os.urandom(self.size))
        while self.is_open:
            os.lseek(self.fp, random.randint(0, self.sz-self.size), 0)
            os.write(self.fp, k)
            os.fsync(self.fp)
            self.io += 1
            
    def random_reads(self):
        while self.is_open:
            os.lseek(self.fp, random.randint(0, self.sz-self.size), 0)
            os.read(self.fp, self.size)
            self.io += 1
            
    def seq_reads(self):
        while self.is_open:
            x = os.read(self.fp, self.size)
            if x == 0:
                os.lseek(self.fp, 0, 0)
            self.io += 1
            
    def seq_writes(self):
        k = bytearray(os.urandom(self.size))
        while self.is_open:
            os.write(self.fp, k)
            os.fsync(self.fp)
            self.io += 1

    def _benchmark(self, fn):
        for power in range(8, 25):
            size = 1 << power
            self.size = size
            self._setup()
            print("benchmarking for size", self.size)
            
            t1 = threading.Thread(target=self.compute_throughput)
            t2 = threading.Thread(target=self.random_writes)

            t1.start()
            t2.start()

            t1.join()
            t2.join()
            
            print(size, sum(self.benchmarks[self.size])/ len(self.benchmarks[self.size]))

    def benchmark_rw(self):
        print("random_writes")
        self._benchmark(self.random_writes)
        os.close(self.fp)
        
    def benchmark_rr(self):
        print("random_reads")
        self._benchmark(self.random_reads)
        os.close(self.fp)
        
    def benchmark_sr(self):
        print("seq_reads")
        self._benchmark(self.seq_reads)
        os.close(self.fp)
        
    def benchmark_sw(self):
        print("seq_writes")
        self._benchmark(self.seq_writes)
        os.close(self.fp)

In [2]:
bm = DiskIOBenchmark()
bm.benchmark_sw()

seq_writes
benchmarking for size 256
256 15431.0
benchmarking for size 512
512 16768.4
benchmarking for size 1024
1024 17412.9
benchmarking for size 2048
2048 16426.2
benchmarking for size 4096
4096 15872.7
benchmarking for size 8192
8192 14045.2
benchmarking for size 16384
16384 11964.5
benchmarking for size 32768
32768 8139.3
benchmarking for size 65536
65536 3763.6
benchmarking for size 131072
131072 3604.9
benchmarking for size 262144
262144 1935.8
benchmarking for size 524288
524288 1323.1
benchmarking for size 1048576
1048576 736.1
benchmarking for size 2097152
2097152 353.4
benchmarking for size 4194304
4194304 196.7
benchmarking for size 8388608
8388608 99.3
benchmarking for size 16777216
16777216 47.8


In [3]:
bm.benchmarks

defaultdict(list,
            {256: [11934,
              16848,
              13479,
              15743,
              17265,
              16822,
              15449,
              17695,
              12576,
              16499],
             512: [15390,
              17437,
              14902,
              17110,
              17613,
              16871,
              17668,
              15127,
              17881,
              17685],
             1024: [16821,
              17815,
              17614,
              16020,
              18426,
              17249,
              17570,
              17444,
              17361,
              17809],
             2048: [16723,
              17174,
              17086,
              16167,
              15393,
              15188,
              15938,
              16208,
              17410,
              16975],
             4096: [14965,
              16799,
              16970,
              14970,
              15697,
     